In [2]:
import pandas as pd
import numpy as np
from numpy import vectorize as vec
import scipy as sp
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

"""
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c rdkit rdkit python=3.7
"""

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors,PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors

In [3]:
mols = pd.read_csv('3CL_enzymatic_activity-clean.tsv', sep='\t', index_col=0)
mols.head(3)

washed_SMILES  \
SAMPLE_ID                                                            
NCGC00093656-03  Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...   
NCGC00262958-01  Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...   
NCGC00162076-02  BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...   

                 mean_pAC50     Class                      PRIMARY_MOA  \
SAMPLE_ID                                                                
NCGC00093656-03         NaN  Inactive  Reverse Transcriptase Inhibitor   
NCGC00262958-01         NaN  Inactive                              NaN   
NCGC00162076-02         NaN  Inactive                              NaN   

                 range_pAC50  count_IDs      SAMPLE_NAME  PUBCHEM_SID  \
SAMPLE_ID                                                               
NCGC00093656-03          NaN          1        Brivudine   90341813.0   
NCGC00262958-01          NaN          1         Bravavir  170466041.0   
NCGC00162076-02          NaN          1  NCGC00162076-02   90340731.0   

                                                 LIBRARY  \
SAMPLE_ID                                                  
NCGC00093656-03                          Anti-infectives   
NCGC00262958-01          Approved Drugs Collection (NPC)   
NCGC00162076-02  Annotated/Bioactive Compound Collection   

                             ASSAY_NAME CURVE_CLASS2 EFFICACY  \
SAMPLE_ID                                                       
NCGC00093656-03  3CL enzymatic activity          4.0      0.0   
NCGC00262958-01  3CL enzymatic activity          4.0      0.0   
NCGC00162076-02  3CL enzymatic activity          4.0      0.0   

                                                            SMILES  \
SAMPLE_ID                                                            
NCGC00093656-03  Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...   
NCGC00262958-01  Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...   
NCGC00162076-02  BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...   

                                                        ALL-SMILES  
SAMPLE_ID                                                           
NCGC00093656-03  Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...  
NCGC00262958-01  Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...  
NCGC00162076-02  BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...

In [4]:
ccrf = mols[['washed_SMILES', 'Class']]
ccrf['Class'] = ccrf['Class'] == 'Active'
ccrf.head(3)

<ipython-input-4-aed672d7a984>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccrf['Class'] = ccrf['Class'] == 'Active'


washed_SMILES  Class
SAMPLE_ID                                                                
NCGC00093656-03  Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...  False
NCGC00262958-01  Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...  False
NCGC00162076-02  BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...  False

In [5]:
Chem.PandasTools.AddMoleculeColumnToFrame(ccrf, smilesCol='washed_SMILES',
molCol='ROMol')

RDKit ERROR: [16:57:39] SMILES Parse Error: syntax error while parsing: [i+]1c2c(c3c1cccc3)cccc2
RDKit ERROR: [16:57:39] SMILES Parse Error: Failed parsing SMILES '[i+]1c2c(c3c1cccc3)cccc2' for input: '[i+]1c2c(c3c1cccc3)cccc2'
/Users/tomohiro-ni/opt/anaconda3/lib/python3.8/site-packages/rdkit/Chem/PandasTools.py:411: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(Chem.MolFromSmiles)


In [7]:
names = [x[0] for x in Descriptors._descList]
print("Number of descriptors in the rdkit: ", len(names))
np.array(names)

Number of descriptors in the rdkit:  200


array(['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex',
       'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt',
       'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons',
       'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge',
       'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2',
       'FpDensityMorgan3', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n',
       'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n',
       'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1',
       'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10',
       'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14',
       'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6',
       'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10',
       'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6',
       'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10',
       'SlogP_VSA11', 'SlogP_VSA12', 'SlogP_VSA2', 

In [8]:
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(names)
from collections import OrderedDict
desc = OrderedDict()
for mol in ccrf.index:
    desc[mol] = calculator.CalcDescriptors(ccrf.loc[mol, 'ROMol'])
desc_mols = pd.DataFrame.from_dict(desc, orient='index', columns=names)

Traceback (most recent call last):
  File "/Users/tomohiro-ni/opt/anaconda3/lib/python3.8/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/Users/tomohiro-ni/opt/anaconda3/lib/python3.8/site-packages/rdkit/Chem/EState/EState.py", line 78, in MaxEStateIndex
    return max(EStateIndices(mol, force))
  File "/Users/tomohiro-ni/opt/anaconda3/lib/python3.8/site-packages/rdkit/Chem/EState/EState.py", line 47, in EStateIndices
    nAtoms = mol.GetNumAtoms()
AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'
Traceback (most recent call last):
  File "/Users/tomohiro-ni/opt/anaconda3/lib/python3.8/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/Users/tomohiro-ni/opt/anaconda3/lib/python3.8/site-packages/rdkit/Chem/EState/EState.py", line 85, in MinEStateIndex
    return min(EStateIndices(mol, force))
  File "/Users/tomohiro-ni/opt/anaconda3/lib/

In [10]:
desc_mols.to_csv('3CL_descriptors.tsv', sep=' ')

In [11]:
desc_mols = (desc_mols - desc_mols.mean())/ desc_mols.std()

In [12]:
desc_mols = desc_mols.dropna(axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(desc_mols, ccrf.Class,
train_size=0.9, test_size=0.1, random_state=0)
print("Training Data")
print("Number of active molecules: ", list(y_train).count(1))
print("Number of inactive molecules: ", list(y_train).count(0))
print("Test Data")
print("Number of active molecules: ", list(y_test).count(1))
print("Number of inactive molecules: ", list(y_test).count(0))

Training Data
Number of active molecules:  274
Number of inactive molecules:  8244
Test Data
Number of active molecules:  37
Number of inactive molecules:  910


In [14]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

       False       0.96      1.00      0.98       910
        True       0.20      0.03      0.05        37

    accuracy                           0.96       947
   macro avg       0.58      0.51      0.51       947
weighted avg       0.93      0.96      0.94       947



In [25]:
train_data = pd.concat([X_train, y_train], axis=1)
active_data = pd.concat([train_data[train_data['Class'] == 1]]*(8244//274-1), ignore_index=True)
train_data  = pd.concat([train_data, active_data], ignore_index=True)
train_data =train_data.sample(frac=1).reset_index(drop=True)
len(train_data[train_data['Class'] == 1])

8220

In [28]:
train_data.head()

MaxEStateIndex  MinEStateIndex  ...   fr_urea  Class
0        0.186693       -0.061174  ...  0.004967  False
1        0.217329       -0.236976  ...  0.004967   True
2        0.346929       -0.398044  ...  0.004967   True
3        0.180023        0.167567  ...  0.004967  False
4        0.086538        0.082823  ...  0.004967  False

[5 rows x 197 columns]

In [26]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(train_data.iloc[:, :-1], train_data.Class)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

       False       0.96      0.99      0.98       910
        True       0.12      0.03      0.04        37

    accuracy                           0.95       947
   macro avg       0.54      0.51      0.51       947
weighted avg       0.93      0.95      0.94       947



In [23]:
from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', 
                         learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                         random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [24]:
nn_model.fit(train_data.iloc[:, :-1], train_data.Class)
print(classification_report(y_test, nn_model.predict(X_test)))

              precision    recall  f1-score   support

       False       0.96      0.97      0.97       910
        True       0.11      0.08      0.09        37

    accuracy                           0.94       947
   macro avg       0.54      0.53      0.53       947
weighted avg       0.93      0.94      0.93       947

